In [1]:
import sys,os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
#from pathlib import Path

from ctypes import *
import math
#cv2_root = "/usr/local/lib-bk/python2.7/dist-packages/cv2"
#sys.path.insert(0,cv2_root)
#cv2_lib_avcodec = "/usr/lib/x86_64-linux-gnu/bkavcodec"
#sys.path.insert(0,cv2_lib_avcodec)
time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190328'
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
image_dir = '/data/darknet/python/' + time_str + '_image/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_detection_result/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_XML_DIR = generated_xml_dir

os.system('mkdir -p ' + generated_xml_dir)

images_dir = SEED_IMG_DIR
data_file = '/data/darknet/cfg/coco.data'
cfg_file = '/data/darknet/cfg/yolov3-nob.cfg'
weights_file = '/data/weights/header_int_aaa_yolov3.weights'


seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

In [2]:
def sample(probs):
    s = sum(probs)
    probs = [a/s for a in probs]
    r = random.uniform(0, 1)
    for i in range(len(probs)):
        r = r - probs[i]
        if r <= 0:
            return i
    return len(probs)-1

def c_array(ctype, values):
    arr = (ctype*len(values))()
    arr[:] = values
    return arr

class BOX(Structure):
    _fields_ = [("x", c_float),
                ("y", c_float),
                ("w", c_float),
                ("h", c_float)]

class DETECTION(Structure):
    _fields_ = [("bbox", BOX),
                ("classes", c_int),
                ("prob", POINTER(c_float)),
                ("mask", POINTER(c_float)),
                ("objectness", c_float),
                ("sort_class", c_int)]


class IMAGE(Structure):
    _fields_ = [("w", c_int),
                ("h", c_int),
                ("c", c_int),
                ("data", POINTER(c_float))]

class METADATA(Structure):
    _fields_ = [("classes", c_int),
                ("names", POINTER(c_char_p))]
    
def classify(net, meta, im):
    out = predict_image(net, im)
    res = []
    for i in range(meta.classes):
        res.append((meta.names[i], out[i]))
    res = sorted(res, key=lambda x: -x[1])
    return res

def detect(net, meta, image, thresh=.5, hier_thresh=.5, nms=.45):
    im = load_image(image, 0, 0)
    num = c_int(0)
    pnum = pointer(num)
    predict_image(net, im)
    dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, None, 0, pnum)
    num = pnum[0]
    if (nms): do_nms_obj(dets, num, meta.classes, nms);

    res = []
    for j in range(num):
        for i in range(meta.classes):
            if dets[j].prob[i] > 0:
                b = dets[j].bbox
                res.append((meta.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))
    res = sorted(res, key=lambda x: -x[1])
    free_image(im)
    free_detections(dets, num)
    return res
#lib = CDLL("/home/pjreddie/documents/darknet/libdarknet.so", RTLD_GLOBAL)
lib = CDLL("/data/darknet/libdarknet.so", RTLD_GLOBAL)
lib.network_width.argtypes = [c_void_p]
lib.network_width.restype = c_int
lib.network_height.argtypes = [c_void_p]
lib.network_height.restype = c_int

predict = lib.network_predict
predict.argtypes = [c_void_p, POINTER(c_float)]
predict.restype = POINTER(c_float)

set_gpu = lib.cuda_set_device
set_gpu.argtypes = [c_int]

make_image = lib.make_image
make_image.argtypes = [c_int, c_int, c_int]
make_image.restype = IMAGE

get_network_boxes = lib.get_network_boxes
get_network_boxes.argtypes = [c_void_p, c_int, c_int, c_float, c_float, POINTER(c_int), c_int, POINTER(c_int)]
get_network_boxes.restype = POINTER(DETECTION)

make_network_boxes = lib.make_network_boxes
make_network_boxes.argtypes = [c_void_p]
make_network_boxes.restype = POINTER(DETECTION)

free_detections = lib.free_detections
free_detections.argtypes = [POINTER(DETECTION), c_int]

free_ptrs = lib.free_ptrs
free_ptrs.argtypes = [POINTER(c_void_p), c_int]

network_predict = lib.network_predict
network_predict.argtypes = [c_void_p, POINTER(c_float)]

reset_rnn = lib.reset_rnn
reset_rnn.argtypes = [c_void_p]

load_net = lib.load_network
load_net.argtypes = [c_char_p, c_char_p, c_int]
load_net.restype = c_void_p

do_nms_obj = lib.do_nms_obj
do_nms_obj.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

do_nms_sort = lib.do_nms_sort
do_nms_sort.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

free_image = lib.free_image
free_image.argtypes = [IMAGE]

letterbox_image = lib.letterbox_image
letterbox_image.argtypes = [IMAGE, c_int, c_int]
letterbox_image.restype = IMAGE

load_meta = lib.get_metadata
lib.get_metadata.argtypes = [c_char_p]
lib.get_metadata.restype = METADATA

load_image = lib.load_image_color
load_image.argtypes = [c_char_p, c_int, c_int]
load_image.restype = IMAGE

rgbgr_image = lib.rgbgr_image
rgbgr_image.argtypes = [IMAGE]

predict_image = lib.network_predict_image
predict_image.argtypes = [c_void_p, IMAGE]
predict_image.restype = POINTER(c_float)

net = load_net(cfg_file, weights_file, 0)
meta = load_meta(data_file)

images = os.listdir(images_dir)

for image in images:
    ret = image.endswith('.jpg')
    if ret == True:
        image_file = images_dir + image
    r = detect(net, meta, image_file)

    origimg = cv2.imread(image_file)
    height = origimg.shape[0]
    width = origimg.shape[1]
    depth = origimg.shape[2]

    xml_str = "<annotation>\n\t\
    <folder>image</folder>\n\t\
    <filename>" + image + "</filename>\n\t\
    " + "<path>" + image_file + "</path>\n\t\
    <source>\n\t\t\
    <database>Unknown</database>\n\t\
    </source>\n\t\
    <size>\n\t\t\
    <width>" + str(width) + "</width>\n\t\t\
    <height>" + str(height) + "</height>\n\t\t\
    <depth>" + str(depth) + "</depth>\n\t\
    </size>\n\t\
    <segmented>0</segmented>"
    #print(xml_str)
    found_flag = False
    for i in range(len(r)):
        cls = r[i][0]
        #if(cls not in ['bus','car','truck', 'motorbike','bicycle','person']):
        if(cls not in ['person', 'motorbike','bicycle']):
            continue
        found_flag = True
        score = r[i][1]
        bbox = r[i][2]

        x1 = max(0, (bbox[0] - bbox[2]/2))
        y1 = max(0, (bbox[1] - bbox[3]/2))
        x2 = min(width, (bbox[0] + bbox[2]/2))
        y2 = min(height, (bbox[1] + bbox[3]/2))      

        p1 = (int(x1), int(y1))
        p2 = (int(x2), int(y2))

        obj_str = "\n\t\
        <object>\n\t\t\
        <name>" + cls + "</name>\n\t\t\
        <pose>Unspecified</pose>\n\t\t\
        <truncated>0</truncated>\n\t\t\
        <difficult>0</difficult>\n\t\t\
        <bndbox>\n\t\t\t\
        <xmin>" + str(int(x1)) + "</xmin>\n\t\t\t\
        <ymin>" + str(int(y1)) + "</ymin>\n\t\t\t\
        <xmax>" + str(int(x2)) + "</xmax>\n\t\t\t\
        <ymax>" + str(int(y2)) + "</ymax>\n\t\t\
        </bndbox>\n\t\
        </object>"

        xml_str += obj_str

        cv2.rectangle(origimg, p1, p2, (0,255,0))
        p3 = (max(p1[0], 15), max(p1[1], 15))
        title = "%s:%.2f" % (cls, score)
        cv2.putText(origimg, title, p3, cv2.FONT_ITALIC, 0.6, (0, 255, 0), 1)
    xml_str += "\n</annotation>"
    if (not found_flag):
        continue
    fileObject = open(generated_xml_dir + image[:-4] + '.xml', 'w')  
    fileObject.write(xml_str)
    fileObject.close()
    print("Created : " + generated_xml_dir + image[:-4] + '.xml')

Created : /data/darknet/python/20190328_detection_result/00002100356.xml
Created : /data/darknet/python/20190328_detection_result/00002100802.xml
Created : /data/darknet/python/20190328_detection_result/00002100052.xml
Created : /data/darknet/python/20190328_detection_result/00002100927.xml
Created : /data/darknet/python/20190328_detection_result/00002100028.xml
Created : /data/darknet/python/20190328_detection_result/00002100511.xml
Created : /data/darknet/python/20190328_detection_result/00002100690.xml
Created : /data/darknet/python/20190328_detection_result/00002100547.xml
Created : /data/darknet/python/20190328_detection_result/00002100681.xml
Created : /data/darknet/python/20190328_detection_result/00002100186.xml
Created : /data/darknet/python/20190328_detection_result/00002100594.xml
Created : /data/darknet/python/20190328_detection_result/00002100235.xml
Created : /data/darknet/python/20190328_detection_result/00002100373.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100310.xml
Created : /data/darknet/python/20190328_detection_result/00002100451.xml
Created : /data/darknet/python/20190328_detection_result/00002100695.xml
Created : /data/darknet/python/20190328_detection_result/00002100013.xml
Created : /data/darknet/python/20190328_detection_result/00002100659.xml
Created : /data/darknet/python/20190328_detection_result/00002100348.xml
Created : /data/darknet/python/20190328_detection_result/00002100723.xml
Created : /data/darknet/python/20190328_detection_result/00002100896.xml
Created : /data/darknet/python/20190328_detection_result/00002100573.xml
Created : /data/darknet/python/20190328_detection_result/00002100620.xml
Created : /data/darknet/python/20190328_detection_result/00002100768.xml
Created : /data/darknet/python/20190328_detection_result/00002100353.xml
Created : /data/darknet/python/20190328_detection_result/00002100300.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100470.xml
Created : /data/darknet/python/20190328_detection_result/00002100864.xml
Created : /data/darknet/python/20190328_detection_result/00002100109.xml
Created : /data/darknet/python/20190328_detection_result/00002100244.xml
Created : /data/darknet/python/20190328_detection_result/00002100148.xml
Created : /data/darknet/python/20190328_detection_result/00002100138.xml
Created : /data/darknet/python/20190328_detection_result/00002100861.xml
Created : /data/darknet/python/20190328_detection_result/00002100103.xml
Created : /data/darknet/python/20190328_detection_result/00002100713.xml
Created : /data/darknet/python/20190328_detection_result/00002100545.xml
Created : /data/darknet/python/20190328_detection_result/00002100486.xml
Created : /data/darknet/python/20190328_detection_result/00002100047.xml
Created : /data/darknet/python/20190328_detection_result/00002100432.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100000.xml
Created : /data/darknet/python/20190328_detection_result/00002100629.xml
Created : /data/darknet/python/20190328_detection_result/00002100559.xml
Created : /data/darknet/python/20190328_detection_result/00002100546.xml
Created : /data/darknet/python/20190328_detection_result/00002100263.xml
Created : /data/darknet/python/20190328_detection_result/00002100628.xml
Created : /data/darknet/python/20190328_detection_result/00002100260.xml
Created : /data/darknet/python/20190328_detection_result/00002100544.xml
Created : /data/darknet/python/20190328_detection_result/00002100379.xml
Created : /data/darknet/python/20190328_detection_result/00002100137.xml
Created : /data/darknet/python/20190328_detection_result/00002100124.xml
Created : /data/darknet/python/20190328_detection_result/00002100856.xml
Created : /data/darknet/python/20190328_detection_result/00002100563.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100726.xml
Created : /data/darknet/python/20190328_detection_result/00002100035.xml
Created : /data/darknet/python/20190328_detection_result/00002100860.xml
Created : /data/darknet/python/20190328_detection_result/00002100609.xml
Created : /data/darknet/python/20190328_detection_result/00002100216.xml
Created : /data/darknet/python/20190328_detection_result/00002100232.xml
Created : /data/darknet/python/20190328_detection_result/00002100825.xml
Created : /data/darknet/python/20190328_detection_result/00002100146.xml
Created : /data/darknet/python/20190328_detection_result/00002100329.xml
Created : /data/darknet/python/20190328_detection_result/00002100730.xml
Created : /data/darknet/python/20190328_detection_result/00002100086.xml
Created : /data/darknet/python/20190328_detection_result/00002100677.xml
Created : /data/darknet/python/20190328_detection_result/00002100816.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100575.xml
Created : /data/darknet/python/20190328_detection_result/00002100386.xml
Created : /data/darknet/python/20190328_detection_result/00002100343.xml
Created : /data/darknet/python/20190328_detection_result/00002100468.xml
Created : /data/darknet/python/20190328_detection_result/00002100736.xml
Created : /data/darknet/python/20190328_detection_result/00002100409.xml
Created : /data/darknet/python/20190328_detection_result/00002100852.xml
Created : /data/darknet/python/20190328_detection_result/00002100836.xml
Created : /data/darknet/python/20190328_detection_result/00002100155.xml
Created : /data/darknet/python/20190328_detection_result/00002100603.xml
Created : /data/darknet/python/20190328_detection_result/00002100624.xml
Created : /data/darknet/python/20190328_detection_result/00002100803.xml
Created : /data/darknet/python/20190328_detection_result/00002100647.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100383.xml
Created : /data/darknet/python/20190328_detection_result/00002100075.xml
Created : /data/darknet/python/20190328_detection_result/00002100924.xml
Created : /data/darknet/python/20190328_detection_result/00002100885.xml
Created : /data/darknet/python/20190328_detection_result/00002100602.xml
Created : /data/darknet/python/20190328_detection_result/00002100774.xml
Created : /data/darknet/python/20190328_detection_result/00002100057.xml
Created : /data/darknet/python/20190328_detection_result/00002100041.xml
Created : /data/darknet/python/20190328_detection_result/00002100301.xml
Created : /data/darknet/python/20190328_detection_result/00002100622.xml
Created : /data/darknet/python/20190328_detection_result/00002100199.xml
Created : /data/darknet/python/20190328_detection_result/00002100473.xml
Created : /data/darknet/python/20190328_detection_result/00002100777.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100157.xml
Created : /data/darknet/python/20190328_detection_result/00002100139.xml
Created : /data/darknet/python/20190328_detection_result/00002100560.xml
Created : /data/darknet/python/20190328_detection_result/00002100485.xml
Created : /data/darknet/python/20190328_detection_result/00002100095.xml
Created : /data/darknet/python/20190328_detection_result/00002100194.xml
Created : /data/darknet/python/20190328_detection_result/00002100753.xml
Created : /data/darknet/python/20190328_detection_result/00002100678.xml
Created : /data/darknet/python/20190328_detection_result/00002100845.xml
Created : /data/darknet/python/20190328_detection_result/00002100721.xml
Created : /data/darknet/python/20190328_detection_result/00002100077.xml
Created : /data/darknet/python/20190328_detection_result/00002100188.xml
Created : /data/darknet/python/20190328_detection_result/00002100921.xml
Created : /data/darknet/python/20190328_detection_r

Created : /data/darknet/python/20190328_detection_result/00002100167.xml
Created : /data/darknet/python/20190328_detection_result/00002100843.xml
Created : /data/darknet/python/20190328_detection_result/00002100152.xml
Created : /data/darknet/python/20190328_detection_result/00002100450.xml
Created : /data/darknet/python/20190328_detection_result/00002100762.xml
Created : /data/darknet/python/20190328_detection_result/00002100071.xml
Created : /data/darknet/python/20190328_detection_result/00002100568.xml
Created : /data/darknet/python/20190328_detection_result/00002100340.xml
Created : /data/darknet/python/20190328_detection_result/00002100865.xml
Created : /data/darknet/python/20190328_detection_result/00002100877.xml
Created : /data/darknet/python/20190328_detection_result/00002100414.xml
Created : /data/darknet/python/20190328_detection_result/00002100784.xml
Created : /data/darknet/python/20190328_detection_result/00002100380.xml
Created : /data/darknet/python/20190328_detection_r

In [5]:
print(len(os.listdir(generated_xml_dir)))

931
